In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from binascii import unhexlify
from pathlib import Path
from common import *
import pickle

DATASET_PATH = "../../tpmscan-dataset"

def load_nonce_measurements(dir):
    results = []

    for path in Path(dir).rglob("Cryptoops_Sign:ECC_*.csv"):
        measurement = str(path).split(os.sep)[-3]
        curve, alg = (str(path).split(os.sep)[-1].split("Cryptoops_Sign:ECC_")[1][:-4]).split("_")
        curve = CURVES_REVERSED[int(curve, 16)]
        alg = ALGS_REVERSED[int(alg, 16)]

        try:
            data = pd.read_csv(path, sep="[,;]", engine="python")
        except:
            print(f"Error reading file: {path}")
            continue

        null_rows = data[["signature_r", "signature_s", "private_key", "digest", "duration"]].isnull().values.any(axis=1)
        if null_rows.sum() > 0:
            data = data[~null_rows]
            if len(data) == 0:
                print(f"Failed collection: {path}")
                continue
            print(f"Warning: Missing {null_rows.sum()} rows ({path})")


        info = get_device_info(Path(path).parent.parent / "results.yaml")

        rep = f"{info['manufacturer']} {info['firmware']} {info['revision']} {alg} {curve} ({path})"

        nonces = []
        for idx, row in data.iterrows():
            r = int(row["signature_r"], 16)
            s = int(row["signature_s"], 16)
            x = int(row["private_key"], 16)
            e = row["digest"]
            if idx == 0:
                pk = reconstruct_pk((int(row["public_key_x"], 16), int(row["public_key_y"], 16)), curve, x)
                try:
                    nonce_point = (int(row["nonce_point_x"], 16), int(row["nonce_point_y"], 16))
                except:
                    nonce_point = None

                rev = info["revision"]
                if (alg == "ECDAA" or alg == "ECSCHNORR") and info["firmware"] == "303.12.0.0":
                    rev = 1.59
                if alg == "ECDAA" and (info["firmware"] == "7.63.13.6400" or info["firmware"] == "5.63.13.6400"): # IFX
                    rev = 1.59
                if alg == "ECSCHNORR" and info["firmware"] == "7.2.0.1": # NTC
                    rev = 1.59
                if alg == "ECSCHNORR" and info["firmware"] == "7.2.0.2": # NTC
                    rev = 1.59
                # ECDAA on NTC 7.2.1.0 seems broken
                # ECDAA on NTC 7.2.2.0 seems broken

                if alg == "ECDAA" and nonce_point is None:
                    print(f"Warning: Missing nonce point for ECDAA (cannot verify) {rep}")
                elif not verify_signature(r, s, pk, e, alg, curve, rev, nonce_point=nonce_point):
                    print(f"Failed verification: {rep}")

            nonces.append(compute_nonce(r, s, x, e, alg, curve, rev))

        secrets = list(map(unhexlify, nonces))
        timings = list(data["duration"] + data["duration_extra"])
        timings_basic = list(data["duration"])
        timings_extra = list(data["duration_extra"])
        nonces = list(map(lambda x: hexlify(x).decode(), secrets))

        results.append({
            "name": measurement,
            "info": info,
            "path": path,
            "curve": curve,
            "alg": alg,
            "secrets": secrets,
            "timings": timings,
            "timings_basic": timings_basic,
            "timings_extra": timings_extra,
            "nonces": nonces,
        })

    return results

measurements = load_nonce_measurements(DATASET_PATH)

In [ ]:
# Save concatenated nonces to nonces/ directory
os.makedirs("nonces", exist_ok=True)
for m in measurements:
    with open(f"nonces/{m['info']['manufacturer']}_{m['info']['firmware']}_{m['alg']}_{m['curve']}_{m['name']}.bin", "wb") as f:
        f.write(b"".join(m["secrets"]))

In [ ]:
# Save timing plots to nonces_plot/ directory
os.makedirs("nonces_plots", exist_ok=True)
for m in measurements:
    for variant in ["full"] + (["basic", "extra"] if m["alg"] == "ECDAA" else []):
        plt.figure(figsize=(10, 10))
        plt.title(f"{m['info']['manufacturer']} {m['info']['firmware']} {m['info']['revision']} {m['alg']} {m['curve']} ({variant}) ({m['name']})")
        plt.xlabel("Nonce MSB")
        plt.ylabel("Time (s)")
        plt.xscale("log", base=2)
        plt.xticks([2 ** x for x in range(0, 9)])
        plt.grid(True, axis="x")

        points = zip(list(map(lambda x: int(x[0]), m["secrets"])), {"full": m["timings"], "basic": m["timings_basic"], "extra": m["timings_extra"]}[variant])
        points = sorted(points, key=lambda x: x[1])
        points = points[:-5]
        points = zip(*points)

        plt.scatter(*points, marker=".", color="#d81b60")
        plt.savefig(f"nonces_plots/{m['info']['manufacturer']}_{m['info']['firmware']}_{m['curve']}_{m['alg']}_{variant}_{m['name']}.png", dpi=300, bbox_inches="tight")
        plt.close()
    break # remove to generate all plots

In [ ]:
# Prepare data for paper plot
paper = []
for m in measurements:
    if m["alg"] == "ECDAA" and m["curve"] == "BN256" and ("c54a5c95a8cca56b" in m["name"] or "91e7840bd977ce81" in m["name"] or "b4b57ab589a6fc0e" in m["name"]):
        paper.append(m)

paper[0], paper[1], paper[2] = paper[1], paper[2], paper[0]

pickle.dump(paper, open("ecc-timing.pickle", "wb"))

In [ ]:
# Paper plot
paper = pickle.load(open("ecc-timing.pickle", "rb"))
fig, axs = plt.subplots(1, 3, gridspec_kw={'width_ratios': [1, 1, 1]}, figsize=(16, 4.4))

i = 0
for m in paper:
    axs[i].set_xlabel(f"Nonce MSB")
    if i == 0:
        axs[i].set_ylabel("Time (s)")
    axs[i].set_xscale("log", base=2)
    axs[i].set_xticks([2 ** x for x in range(0, 9)])
    axs[i].grid(True, axis="x")
    axs[i].set_title(f"{m['info']['manufacturer']} {m['info']['firmware']}", fontsize=12, y=-0.25)

    points = zip(list(map(lambda x: int(x[0]), m["secrets"])), m["timings"])
    points = sorted(points, key=lambda x: x[1])
    points = points[:-5]
    points = zip(*points)

    axs[i].scatter(*points, marker=".", color="#d81b60")

    if i == 0:
        axs[i].axhline(0.10125, 0, color="lightgray", linestyle="dashed")
    if i == 1:
        axs[i].axhline(0.21125, 0, color="lightgray", linestyle="dashed")
    if i == 2:
        axs[i].axhline(0.0655, 0, color="lightgray", linestyle="dashed")

    i += 1


plt.savefig("ecc-timing.pdf", dpi=600, bbox_inches="tight")
plt.show()